## **Calculations for non-integrated LCA**

📍 **World activity calculations are performed to get breakdown results down to the first level**

📍 **Regional activity calculations are performed to get total scores**

In [ ]:
from brightway2 import *
import bw2data as bd
import pyprind
from pathlib import Path
import pandas as pd

In [ ]:
projects.set_current("Integrated LCA project") #<---- Project name
databaseNames = databases
myDatabaseNames = []
for databaseName in databaseNames:
    if'_3.8_' not in databaseName and 'bio' not in databaseName and 'eco' not in databaseName and not 'A_' in databaseName:
        myDatabaseNames.append(databaseName)
myDatabaseNames.sort()

##### **Choose method to run LCIA**

In [ ]:
GWPMethod2021 = [method for method in methods if 'IPCC 2021' in str(method) and  'climate change: including SLCFs' 
                in str(method) and 'GWP 100a, incl. H and bio CO2' in str(method) and not 'no LT' in str(method)]
method = [GWPMethod2021][0]
method

##### **1. Function that calculates the breakdown LCA impacts**

In [ ]:
def breakdown_calculations(db, activity):
    activitiesList = [{activity : 1}]
    for exchange in activity.technosphere():
        activitiesList.append({Database(exchange.input.key[0]).get(exchange.input.key[1]) : exchange.amount})
    calculationSetup = {'inv' : activitiesList, 'ia' : method}  #<---- Change the method when you need to
    calculation_setups['breakdown'] = calculationSetup
    myLCA = MultiLCA('breakdown')
    results = pd.DataFrame(myLCA.results.transpose(), columns = [str(list(i.keys())[0]).split('\'')[1] for i in activitiesList], index = pd.MultiIndex.from_tuples(method))
    results = results.sort_index().drop(index = [i for i in results.index if i[0] == 'rest'])
    directEmissions = pd.DataFrame([0 if abs(results.iloc[r,0] - results.iloc[r,1:].sum()) / abs(results.iloc[r,0]) < 1e-5 else results.iloc[r, 0] - results.iloc[r, 1:].sum() for r in range(len(results.index))],
                                      columns = ['direct emissions'],
                                      index = results.index)
    results = pd.concat([results, directEmissions], axis = 1)
    results['database'] = db.name
    results['location'] = activity['location']
    return results

In [ ]:
result_dataframes = {
    'diesel DAC wind' : pd.DataFrame(),
    'diesel DAC poplar' : pd.DataFrame(),
}

In [ ]:
for db in pyprind.prog_percent(myDatabaseNames):
        get_db = bd.Database(db)

        diesel_act = [act for act in get_db if 'transport, freight, lorry' in act['name'] and 'diesel' in act['name']
                                and 'World' in act['location']]
        for i in diesel_act:
                if "DAC" in i['name'] and "wind" in i['name']:
                        result_dataframes['diesel DAC wind'] = pd.concat([result_dataframes['diesel DAC wind'],
                                breakdown_calculations(get_db, i)],
                                ignore_index = True
                                )
                if "DAC" in i['name'] and "poplar" in i['name']:
                        result_dataframes['diesel DAC poplar'] = pd.concat([result_dataframes['diesel DAC poplar'],
                                breakdown_calculations(get_db, i)],
                                ignore_index = True
                                )

- Renaming columns so we can sum them

In [ ]:
for i in result_dataframes:
    df = result_dataframes[i]
    for col in df.columns:
        if "transport, freight, lorry, diesel, 40t gross weight" in col:
            df.rename(columns = {str(col): "transport, diesel truck "}, inplace = True) 
        if "Heavy duty truck" in col:
            df.rename(columns = {str(col): "Heavy duty truck"}, inplace = True)



for i in result_dataframes:
    df = result_dataframes[i]
    df = df.fillna(0)
    
    df_new = pd.DataFrame()
    df_tr = pd.DataFrame()
    
    for col in df.columns:
        if 'transport, ' in col:
            df_new = df[col].to_frame()
            if 'transport, diesel truck' in col:
                df.rename(columns={col: 'Total'}, inplace=True)
        if 'Heavy duty truck' in col:
            df_tr = df[col].to_frame()
        if 'market for diesel,' in col:
            df.rename(columns={col: 'diesel'}, inplace=True)


    if not df_new.empty:
        df_tot = df_new.sum(axis=1)
        col_name = df_new.columns[0]
        df_tot = df_tot.to_frame(name=col_name)
    else:
        df_tot = pd.DataFrame()

    if not df_tr.empty:
        df_truck = df_tr.sum(axis=1)
        col_name = df_tr.columns[0]
        df_truck = df_truck.to_frame(name=col_name)
    else:
        df_truck = pd.DataFrame()

    columns_to_drop = []
    for col in df.columns:
        if "transport, " in col: 
            columns_to_drop.append(col)
        if "Heavy duty truck" in col: 
            columns_to_drop.append(col)
        if "treatment" in col: 
            columns_to_drop.append(col)
    columns_to_drop = set(columns_to_drop)

    for column in columns_to_drop:
        if column in df.columns:
            df.drop(column, axis=1, inplace=True)
    
    # Create the final dataframe
    dfs_toadd = [df_tot, df_truck, df]
    df_final = pd.concat(dfs_toadd, axis=1)
    
    # Replace the calculated dataframe with the modified
    result_dataframes[i] = df_final
    df = result_dataframes[i]

In [ ]:
dieselFileName = 'diesel breakdown results regional.xlsx'

import openpyxl 
import os

def check_or_create_excel_file(filePath):
    if not os.path.exists(filePath):
        wb = openpyxl.Workbook()
        wb.save(filePath)
        print(f"Excel file created at {filePath}")

def write_dataframes_to_excel(dictionary, file_name):
    check_or_create_excel_file(file_name)
    with pd.ExcelWriter(file_name, engine = 'openpyxl', mode = 'a') as writer:        
        for sheet_name, dataframe in dictionary.items():
            if sheet_name in writer.book.sheetnames:
                writer.book.remove(writer.book[sheet_name])
            dataframe.to_excel(writer, sheet_name = sheet_name)
    wb = openpyxl.load_workbook(file_name)
    if wb.sheetnames[0] == 'Sheet':  # check if the workbook has any sheets
        firstSheet = wb.sheetnames[0]  # get the name of the first sheet
        wb.remove(wb[firstSheet])  # remove the first sheet
        wb.save(file_name)

In [ ]:
diesel_dataframes = {}
for key in list(result_dataframes.keys()):
    if 'diesel' in key:
        diesel_dataframes[key] = result_dataframes[key]

* Write dataframes to excel 

📍 This creates a folder called "Results" in the same directory

In [ ]:
write_dataframes_to_excel(diesel_dataframes, os.path.join('Results', 'IPCC 2021','Diesel', dieselFileName))

#### **2. Calculation of all activities (also regional) - Total scores**

In [ ]:
acts_get =['transport, freight, lorry, diesel, 40t gross weight,']
db_store = []
for i in databases:
    if "ecoinvent_cutoff_3.8_remind" in i: 
        db_store.append(i)
db_store

In [ ]:
def multiLCA_calculations(db, FUs: list, methods: list): #<---- Activity list as dictionary and method list
    acts= []
    for i in FUs:
        names = {i:1}
        acts.append(names)

    calculationSetup = {'inv' : acts, 'ia' : methods}  #<---- Change the method when you need to
    calculation_setups['breakdown'] = calculationSetup
    multilca = MultiLCA('breakdown')  
    df = pd.DataFrame(multilca.results, columns=[met  for met in methods])
    df.insert(0, "Activity name", [act['name'] for act in FUs])
    df.insert(1, "Database", db.split("_")[4])
    df.insert(2, "Year", db.split("_")[-1])
    df.insert(3, "Location", [act['location'] for act in FUs])
    
    return df

- Function for mapping activities 

In [ ]:
def classifiy_ds_names(df):
    df.loc[df["Activity name"].str.contains("diesel"), "Fuel"] = "Diesel"
    df.loc[df["Activity name"].str.contains("wind"), "hydrogen"] = "Wind"
    df.loc[(df["Activity name"].str.contains("poplar")), "hydrogen"] = "biomass w/CCS"
    df.loc[(df["Activity name"].str.contains("biomass")), "hydrogen"] = "biomass w/CCS"
    df.loc[df["Activity name"].str.contains("DAC"), "CO2"] = "DAC"
    df.loc[df["Activity name"].str.contains("dioxide"), "CO2"] = "post"

    return df

- Calculations

In [ ]:
storing = []
for db in pyprind.prog_percent(db_store): 
    get_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    get_diesel = [act for act in get_db if "diesel" in act['name'] and "transport" in act['name'] and any(x in act['name'].lower() for x in ["DAC", "wind", "poplar", "dioxide"])]

    trucks = get_diesel
    df = multiLCA_calculations(db, trucks, method)
    list_df = [df]
    storing = storing + list_df

df_STORE_all = pd.concat(storing, axis=0)
df_STORE_all = df_STORE_all.reset_index(drop=True)
df_STORE = classifiy_ds_names(df_STORE_all)
df_STORE.to_excel("Total_score_regional_nonIntegrated.xlsx")

##### **3. Technology sensitivity**

In [ ]:
def classifiy_ds_names(df):
    df.loc[df["Activity name"].str.contains("DAC"), "CO2"] = "DAC"
    df.loc[df["Activity name"].str.contains("direct air capture"), "CO2"] = "DAC"
    df.loc[df["Activity name"].str.contains("post-combustion"), "CO2"] = "post-combustion"
    df.loc[df["Activity name"].str.contains("electrolysis"), "hydrogen"] = "electrolysis"
    df.loc[(df["Activity name"].str.contains("poplar")), "hydrogen"] = "poplar"
    df.loc[(df["Activity name"].str.contains("gasification of woody biomass")), "hydrogen"] = "gasification - biomass"
    df.loc[(df["Activity name"].str.contains("hard coal gasification")), "hydrogen"] = "gasification - hard coal"
    df.loc[df["Activity name"].str.contains("mix"), "electricity"] = "grid"
    df.loc[df["Activity name"].str.contains("1-3MW turbine_ offshore"), "electricity"] = "wind - offshore - 1-3MW"
    df.loc[df["Activity name"].str.contains("1-3MW turbine_ onshore"), "electricity"] = "wind - onshore - 1-3MW"
    df.loc[df["Activity name"].str.contains(">3MW turbine"), "electricity"] = "wind - 3MW"
    df.loc[df["Activity name"].str.contains("nuclear"), "electricity"] = "nuclear"
    df.loc[df["Activity name"].str.contains("_hydro_ pumped storage"), "electricity"] = "hydro, pumped storage"
    df.loc[df["Activity name"].str.contains("_hydro_ reservoir_ alpine region"), "electricity"] = "hydro, reservoir, alpine"
    df.loc[df["Activity name"].str.contains("_hydro_ reservoir_ non-alpine region"), "electricity"] = "hydro, reservoir, non-alpine"
    df.loc[df["Activity name"].str.contains("_hydro_ run-of-river"), "electricity"] = "hydro, run-of-river"
    df.loc[df["Activity name"].str.contains("solar"), "electricity"] = "solar"
    df.loc[df["Activity name"].str.contains("Alkaline"), "electrolyzer"] = "Alkaline"
    df.loc[df["Activity name"].str.contains("PEM"), "electrolyzer"] = "PEM"
    df.loc[df["Activity name"].str.contains("waste heat"), "heat"] = "waste"
    df.loc[df["Activity name"].str.contains("industrial steam heat"), "heat"] = "industrial steam heat"
    df.loc[df["Activity name"].str.contains("heat pump heat"), "heat"] = "heat pump heat"
    df.loc[df["Activity name"].str.contains("CCS"), "CCS"] = True

    return df

In [ ]:
db_store = []
for i in databases:
    if "ecoinvent_cutoff_3.8_remind" in i: 
        db_store.append(i)
acts_get =['transport, freight, lorry, diesel, 40t gross weight,']

In [ ]:
storing = []
for db in pyprind.prog_percent(db_store): 
    get_db = [bd.Database(db_tr) for db_tr in databases if db_tr == db.split("_3.8_")[1]][0]
    get_diesel = [act for act in get_db if "transport" in act['name'] and any(x in act['name'] for x in ["DAC", "post"])]
    trucks = get_diesel
    df = multiLCA_calculations(db, trucks, method)
    list_df = [df]
    storing = storing + list_df

df_STORE_all = pd.concat(storing, axis=0)
df_STORE_all = df_STORE_all.reset_index(drop=True)
df_STORE = classifiy_ds_names(df_STORE_all)
df_STORE.to_excel("Total_score_techsens_nonIntegrated.xlsx")